# Consultas Vectoriales en LanceDB con el Dataset TMDb

Este notebook utiliza el dataset **TMDb 5000 Movies** para realizar consultas vectoriales usando descripciones de películas.

## Paso 1: Cargar el Dataset


In [12]:
import pandas as pd
import os
from sentence_transformers import SentenceTransformer
import numpy as np
import pyarrow as pa
import lancedb 

In [3]:
# Ruta al archivo
data_path = "data/tmdb_5000_movies.csv"

# Cargar el dataset
movies = pd.read_csv(data_path)

# Mantener solo las columnas necesarias
movies = movies[["id", "title", "overview", "genres"]]

# Reemplazar valores nulos en la descripción
movies["overview"] = movies["overview"].fillna("")

# Mostrar un resumen del dataset
print(movies.head())

       id                                     title  \
0   19995                                    Avatar   
1     285  Pirates of the Caribbean: At World's End   
2  206647                                   Spectre   
3   49026                     The Dark Knight Rises   
4   49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                              genres  
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...  
1  [{"id": 12, "name": "Adventure"}, {"id": 14, "...  
2  [{"id": 28, "name": "Action"}, {"id": 12, "nam...  
3  [{"id": 28, "name": "Action"}, {"id": 80, "nam...  
4  [{"id": 28, "name": "Action"}, {"id": 12, "nam...  


## Paso 2: Generar Embeddings de las Descripciones
Utilizaremos el modelo **all-MiniLM-L6-v2** de `sentence-transformers` para convertir las descripciones en vectores numéricos.

In [4]:
# Cargar el modelo preentrenado
text_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generar embeddings para las descripciones
movies["embedding"] = movies["overview"].apply(lambda x: text_model.encode(x).astype(np.float32).tolist())

print("Embeddings generados para las películas.")

Embeddings generados para las películas.


## Paso 3: Guardar los Datos en LanceDB
Conectaremos a LanceDB y almacenaremos los datos con un esquema adecuado.

In [14]:
# Conectar a LanceDB
db = lancedb.connect("data/movies")

# Especificar esquema con id como entero
schema = pa.schema([
    ("id", pa.int64()),  # Cambiado a entero
    ("title", pa.string()),
    ("overview", pa.string()),
    ("genres", pa.string()),
    ("embedding", pa.list_(pa.float32(), list_size=len(movies["embedding"][0])))
])

# Convertir a una tabla Arrow
table = pa.Table.from_pandas(movies, schema=schema)

# Crear la tabla en LanceDB
tabla = db.create_table("movies", schema=schema, mode="overwrite")
tabla.add(table)
print("Tabla creada correctamente.")


Tabla creada correctamente.


[2024-12-02T05:45:20Z WARN  lance::dataset] No existing dataset at /home/jake3120/itam/FuentesDeDatos/LanceDB/notebooks/data/movies/movies.lance, it will be created


## Paso 4: Realizar Consultas Vectoriales
Ejecutaremos múltiples consultas para encontrar películas similares basadas en una descripción textual.

In [15]:
# Consulta 1: Películas de ciencia ficción
query_description_1 = "Thrilling science fiction movie"
query_embedding_1 = text_model.encode(query_description_1).astype(np.float32)

# Realizar búsqueda
results_query_1 = tabla.search(query_embedding_1, vector_column_name="embedding").limit(5).to_pandas()

print("Películas recomendadas para la consulta 1:")
print(results_query_1[["title", "overview"]])

Películas recomendadas para la consulta 1:
                     title                                           overview
0  Da Sweet Blood of Jesus  A movie about human beings who are addicted to...
1      Harrison Montgomery                            Film from Daniel Davila
2                 Fortress  A futuristic prison movie. Protagonist and wif...
3                  Gattaca  Science fiction drama about a future society i...
4              Family Plot  Lighthearted suspense film about a phony psych...


In [16]:
# Consulta 2: Drama romántico
query_description_2 = "A romantic drama about love and loss."
query_embedding_2 = text_model.encode(query_description_2).astype(np.float32)

# Realizar búsqueda
results_query_2 = tabla.search(query_embedding_2, vector_column_name="embedding").limit(5).to_pandas()

print("Películas recomendadas para la consulta 2:")
print(results_query_2[["title", "overview"]])

Películas recomendadas para la consulta 2:
                title                                           overview
0              Closer  A witty, romantic, and very dangerous love sto...
1       The Last Song  A drama centered on a rebellious girl who is s...
2          16 to Life  Romantic comedy. A small town teenager's angst...
3  As It Is in Heaven  A musical romantic tragedy about a famous comp...
4   Darling Companion  The story of a woman who loves her dog more th...
